In [0]:
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = ""

In [0]:
from NN import *
from utils import get_tf_session
from data_utils import make_batch, single_np_datapoint_generator

import tensorflow as tf
import matplotlib.pyplot as plt
import time

In [0]:
train_dataset_files = [
 './data/tfrecords/0_cropped.tfrecord',
 './data/tfrecords/1_cropped.tfrecord',
 './data/tfrecords/2_cropped.tfrecord',
 './data/tfrecords/3_cropped.tfrecord',
 './data/tfrecords/4_cropped.tfrecord',
 './data/tfrecords/5_cropped.tfrecord',
 './data/tfrecords/6_cropped.tfrecord',
 './data/tfrecords/7_cropped.tfrecord',
 './data/tfrecords/8_cropped.tfrecord',
]

validation_dataset_files = ['./data/tfrecords/9_cropped.tfrecord']

In [4]:
X_batch, Y_batch = make_batch(train_dataset_files, 200, raw_values=True)
no_of_outputs = 2
_, gender_logits = get_classification_NN(X_batch, 60, [lambda i: 10 + 3*i], 4, no_of_outputs)
loss = tf.losses.sparse_softmax_cross_entropy(tf.cast(Y_batch[:, 0], tf.int32), gender_logits)
train_op = get_training_op(loss)
X = tf.placeholder(dtype=tf.float32, shape=(None, None, None, 3))
gender, _ = get_classification_NN(X, 60, [lambda i: 10 + 3*i], 4, no_of_outputs)

Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.


In [5]:
sess = get_tf_session()
if tf.test.is_gpu_available():
    print("Tensorflow is able to detect gpu...")

Tensorflow is able to detect gpu...


In [7]:
avg_loss = 0
print_int = 200
st = time.time()
# while datetime.datetime.now().time() < datetime.time(hour=8, minute=30):
for i in range(5000):
    _, loss_value= sess.run([train_op, loss])
    avg_loss += loss_value
    if i>0 and i%print_int == 0:
        print("Batch No.", i, "Avg loss:", avg_loss/print_int)
        avg_loss = 0
print("training took total", time.time() - st, "seconds")

Batch No. 200 Avg loss: 0.24619824014604091
Batch No. 400 Avg loss: 0.23567437753081322
Batch No. 600 Avg loss: 0.2266256430745125
Batch No. 800 Avg loss: 0.2200286468863487
Batch No. 1000 Avg loss: 0.21240913912653922
Batch No. 1200 Avg loss: 0.2047497658059001
Batch No. 1400 Avg loss: 0.19937069818377495
Batch No. 1600 Avg loss: 0.19711781933903694
Batch No. 1800 Avg loss: 0.19487049628049136
Batch No. 2000 Avg loss: 0.1987031790241599
Batch No. 2200 Avg loss: 0.19357699871063233
Batch No. 2400 Avg loss: 0.18799819212406874
training took total 182.07158017158508 seconds


In [8]:
gen = single_np_datapoint_generator(validation_dataset_files, raw_values=True)
correct = 0
total = 0
for img, attr in group_iterable_into_list(gen, 20, 2):
    pred = sess.run([gender], feed_dict={X: img})[0]
    gend = np.array(attr)[:, 0]
    pred_labels = np.argmax(pred, axis=1)
    correct += np.count_nonzero(pred_labels == gend)
    total += pred.shape[0]
print("Accuracy", correct/total)

Accuracy 0.8343313373253493
